In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint
from getpass import getpass

from langchain_openai import ChatOpenAI

from core.state import ReWOO
from core.planner import planner_agent
from core.worker import worker_agent
from langgraph.graph import StateGraph, END

In [4]:
os.environ["OPENAI_API_KEY"] = getpass("OPENAI API Key:")

In [3]:
query = "What is the hometown of the 2024 australian open winner?"

In [5]:
gpt35turbo_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

In [6]:
from core.planner import Planner
from textwrap import dedent

planner = Planner(
    llm=gpt35turbo_llm,
    few_shot=dedent("""
    For example,
    Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
    hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
    less than Toby. How many hours did Rebecca work?
    Plan : Given Thomas worked x hours, translate the problem into algebraic expressions and solve
    with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
    Plan : Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
    Plan : Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]
    """)
)

In [7]:
from langchain_core.tools import BaseTool

class CustomSearchTool(BaseTool):
    name = "Google"
    description = "Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. The input should be a search query."

    def _run(self, input: str):
        return input


class CustomLLMTool(BaseTool):
    name = "LLM"
    description = "A pretrained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction."

    def _run(self, input: str):
        return input


tools = [CustomSearchTool(), CustomLLMTool()]

In [8]:
flow = StateGraph(ReWOO)
flow.add_node("planner_node", planner_agent)
flow.add_node("worker_node", worker_agent)
flow.add_edge("planner_node", "worker_node")
flow.add_edge("worker_node", END)
flow.set_entry_point("planner_node")
app = flow.compile()

In [9]:
# Iterate over the stream of outputs
for output in app.stream(ReWOO(query=query, planner=planner, tools=tools)):
    # Each output is a dictionary where keys are node names and values are outputs
    for node, state_output in output.items():
        pprint(node)
        pprint(state_output["planner"].plans)

'planner_node'
{'#E1': {'args': '2024 Australian Open winner',
         'plan': 'Use Google to search for the 2024 Australian Open winner.',
         'tool_name': 'Google'},
 '#E2': {'args': 'What is the name of the 2024 Australian Open winner, given '
                 '#E1',
         'plan': 'Retrieve the name of the 2024 Australian Open winner from '
                 'the search results.',
         'tool_name': 'LLM'},
 '#E3': {'args': 'hometown of 2024 Australian Open winner, given #E2',
         'plan': 'Use Google to search for the hometown of the 2024 Australian '
                 'Open winner.',
         'tool_name': 'Google'},
 '#E4': {'args': 'What is the hometown of the 2024 Australian Open winner, '
                 'given #E3',
         'plan': 'Retrieve the hometown of the 2024 Australian Open winner '
                 'from the search results.',
         'tool_name': 'LLM'}}
2024 Australian Open winner


TypeError: replace() argument 2 must be str, not None

In [53]:
state_output["planner"].plans

{'#E1': {'plan': 'Search for more information about Melanie C. ',
  'tool_name': 'Google',
  'args': 'Melanie C'},
 '#E2': {'plan': 'Find out who featured on Melanie Cs number one single Never Be The Same Again in 2000.',
  'tool_name': 'LLM',
  'args': 'Who featured on Melanie Cs number one single Never Be The Same Again in\n2000? Given context: #E1 ',
  'evidence': None},
 '#E3': {'plan': 'Search for more information about the featured artist.',
  'tool_name': 'Google',
  'args': ' #E2 '},
 '#E4': {'plan': 'Find out if the featured artist died in a car accident in 2002.',
  'tool_name': 'LLM',
  'args': 'Did #E2 die in a car accident in 2002? Given context: #E2 '}}

In [52]:
import re
tools_map = {tool.name: tool for tool in tools}

plans = state_output["planner"].plans
for plan in plans.values():
    args = plan["args"]
    for idx in re.findall(r"#E\d+", plan["args"]):
        if "evidence" in  plans[idx]:
            args = plan["args"].replace(idx, plans[idx]["evidence"])
    plan["evidence"] = tools_map[plan["tool_name"]].invoke(args)

Who featured on Melanie Cs number one single Never Be The Same Again in
2000? Given context: #E1 


TypeError: replace() argument 2 must be str, not None

In [45]:
"evidence" in plans["#E1"]

False